# 🏥 Lab 3: Medical Image Classification with Neural Networks

**AI in Medicine and Healthcare**  
**Insper Instituto de Ensino e Pesquisa**  
**Week 2 - Class 3**

---

## 📝 Student Information (REQUIRED)

**Student 1:**
- Name: _______________________
- Email: ______________________

**Student 2:**
- Name: _______________________
- Email: ______________________

**Date Submitted:** ______________

---

## 🎯 Today's Challenge

**Can we use the neural networks we learned in Lab 2 to diagnose pneumonia from chest X-rays?**

You already know how to:
- ✅ Build neural networks
- ✅ Implement training loops
- ✅ Evaluate with medical metrics

**Today's new skills:**
- 🆕 Load and visualize medical images
- 🆕 Preprocess images for neural networks
- 🆕 Apply your Lab 2 knowledge to a new domain

**The twist:** We'll use the SAME type of network (MLP) we used for tabular data... on images. Will it work? Let's find out!

---

## 📊 Dataset: Chest X-Ray Pneumonia

- **Source:** Kaggle Chest X-Ray Images (Pneumonia)
- **Classes:** Normal vs Pneumonia
- **Images:** ~5,800 chest X-rays from pediatric patients
- **Task:** Binary classification (like Lab 2, but with images!)
- **Clinical Importance:** Pneumonia kills ~2.5 million people/year globally

---

## ⏱️ Time Allocation

- **Part 1:** Image Loading & Visualization (20 min)
- **Part 2:** Build MLP Classifier (30 min)
- **Part 3:** Training & Evaluation (20 min)
- **Part 4:** Reflection & Analysis (20 min)

**Total:** 90 minutes

---

## 📊 Grading

| Components |
|-----------|
| Part 1: Image Processing |
| Part 2: Model Architecture |
| Part 3: Training & Evaluation |
| Part 4: Reflection Questions |

### Assessment

- **Inadequate**: Did not complete all parts
- **Adequate**: Completed all parts
- **Above Average**: Completed all parts and Bonus Exploration

---

# 🚀 PART 1: Image Loading & Visualization (20 minutes)

First, let's see what medical images actually look like!

---

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score, precision_score

import os
from pathlib import Path

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Step 1.1: Download the Dataset

We'll use a subset of the Chest X-Ray dataset for faster training.

In [ ]:
# Download and extract dataset
!gdown --id 17MSlc-0l_SSRevYBbKwQQ1PQGpi_W6VN
!unzip -q chest_xray_compact.zip

print("✓ Dataset ready!")


## Step 1.2: Explore the Dataset Structure

**TODO:** Explore the directory structure and count images in each category.

In [ ]:
# TODO: Set the data directory path
# Hint: data_dir = Path('chest_xray_compact')

data_dir = None



# TODO: Count images in train/val/test for both classes
# Hint: Use os.listdir() or Path().glob('*.jpeg')
# Structure: chest_xray_compact/train/NORMAL/*.jpeg and chest_xray_compact/train/PNEUMONIA/*.jpeg

train_normal_count = None
train_pneumonia_count = None




# Print summary
print("Dataset Structure:")
print("="*50)
print(f"Training set:")
print(f"  Normal: {train_normal_count} images")
print(f"  Pneumonia: {train_pneumonia_count} images")
print(f"  Total: {train_normal_count + train_pneumonia_count} images")
print(f"\nClass balance: {train_pneumonia_count/(train_normal_count + train_pneumonia_count)*100:.1f}% pneumonia")

## Step 1.3: Visualize Sample Images

**TODO:** Load and display sample chest X-rays from both classes.

In [ ]:
# TODO: Load one normal and one pneumonia X-ray
# Hint: Use PIL.Image.open(image_path)
# Paths: chest_xray_compact/train/NORMAL/... and chest_xray_compact/train/PNEUMONIA/...

# Get sample image paths
normal_images = None  # list(Path('chest_xray_compact/train/NORMAL').glob('*.jpeg'))
pneumonia_images = None  # list(Path('chest_xray_compact/train/PNEUMONIA').glob('*.jpeg'))



# Load first image from each class
normal_img = None  # Image.open(normal_images[0])
pneumonia_img = None  # Image.open(pneumonia_images[0])



# TODO: Display the images side by side
# Hint: Use matplotlib subplot

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot normal
# axes[0].imshow(normal_img, cmap='gray')
# axes[0].set_title('Normal Chest X-Ray', fontsize=14, fontweight='bold')
# axes[0].axis('off')



# Plot pneumonia
# axes[1].imshow(pneumonia_img, cmap='gray')
# axes[1].set_title('Pneumonia Chest X-Ray', fontsize=14, fontweight='bold')
# axes[1].axis('off')



plt.tight_layout()
plt.show()

# Print image information
print(f"Normal X-ray size: {normal_img.size if normal_img else 'N/A'}")
print(f"Pneumonia X-ray size: {pneumonia_img.size if pneumonia_img else 'N/A'}")

## Step 1.4: Create Custom Dataset Class

**TODO:** Create a PyTorch Dataset class to load images.

In [ ]:
# TODO: Define a custom Dataset class
# Hint: Inherit from torch.utils.data.Dataset
# Must implement __init__, __len__, and __getitem__

class ChestXRayDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir: Directory with 'NORMAL' and 'PNEUMONIA' subdirectories
            transform: Optional transform to apply to images
        """
        # TODO: Store parameters
        # self.root_dir = root_dir
        # self.transform = transform



        # TODO: Collect all image paths and labels
        # Hint: Loop through NORMAL (label=0) and PNEUMONIA (label=1) folders
        # Store as list of tuples: [(path1, label1), (path2, label2), ...]

        self.images = []  # Will store (image_path, label) tuples

        # Normal images (label = 0)



        # Pneumonia images (label = 1)



    def __len__(self):
        # TODO: Return total number of images

        pass

    def __getitem__(self, idx):
        # TODO: Load image and label at index idx
        # Hint:
        # 1. Get image path and label from self.images[idx]
        # 2. Load image using PIL.Image.open()
        # 3. Convert to RGB if needed
        # 4. Apply transform if provided
        # 5. Return (image_tensor, label)




        pass

# Test the dataset
print("Testing Dataset class...")
# Uncomment when implemented:
# test_dataset = ChestXRayDataset('chest_xray_compact/train')
# print(f"Total images: {len(test_dataset)}")
# sample_img, sample_label = test_dataset[0]
# print(f"Sample image type: {type(sample_img)}")
# print(f"Sample label: {sample_label}")

## Step 1.5: Define Image Transforms

**TODO:** Create transforms to preprocess images for the neural network.

In [ ]:
# TODO: Define image transformations
# Required steps:
# 1. Resize to fixed size (e.g., 64x64 for faster training)
# 2. Convert to grayscale
# 3. Convert to tensor
# 4. Normalize (mean=0.5, std=0.5 for grayscale)

# Hint: Use transforms.Compose([
#     transforms.Resize((64, 64)),
#     transforms.Grayscale(num_output_channels=1),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5], std=[0.5])
# ])

transform = None




print("Image transformations defined:")
print(transform if transform else "TODO: Define transforms")

## Step 1.6: Create DataLoaders

**TODO:** Create train, validation, and test DataLoaders.

In [ ]:
# TODO: Create datasets with transforms
# Hint: train_dataset = ChestXRayDataset('chest_xray_compact/train', transform=transform)

train_dataset = None
val_dataset = None
test_dataset = None




# TODO: Create DataLoaders
# Hint: Use batch_size=32, shuffle=True for train

train_loader = None
val_loader = None
test_loader = None




# Verify
print("DataLoaders created:")
print(f"Train batches: {len(train_loader) if train_loader else 'N/A'}")
print(f"Val batches: {len(val_loader) if val_loader else 'N/A'}")
print(f"Test batches: {len(test_loader) if test_loader else 'N/A'}")

# Test loading a batch
if train_loader:
    images, labels = next(iter(train_loader))
    print(f"\nBatch shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")
    print(f"Image range: [{images.min():.2f}, {images.max():.2f}]")

## 🎉 Part 1 Complete!

**Checkpoint:**
- ✅ Dataset downloaded and explored
- ✅ Sample images visualized
- ✅ Custom Dataset class created
- ✅ Image transforms defined
- ✅ DataLoaders ready

**Key Insight:** Images are 64×64 pixels = 4,096 input features!
(Compare to Lab 2: only 8 features)

---

---

# 🧠 PART 2: Build MLP Classifier (30 minutes)

**The Challenge:** Use the SAME type of network from Lab 2 (MLP) on images.

**Key Question:** A 64×64 grayscale image has 4,096 pixels. How do we feed this to an MLP?

**Answer:** FLATTEN the image into a 1D vector!

---

## Step 2.1: Visualize What Flattening Does

**TODO:** Show what happens when we flatten a 2D image.

In [ ]:
# Get a sample image
if train_loader:
    sample_images, sample_labels = next(iter(train_loader))
    sample_img = sample_images[0]  # Shape: (1, 64, 64)

    # TODO: Flatten the image
    # Hint: flattened = sample_img.view(-1)

    flattened = None



    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    # Original image
    axes[0].imshow(sample_img.squeeze(), cmap='gray')
    axes[0].set_title('Original Image\n(64×64 = 4,096 pixels)', fontweight='bold')
    axes[0].axis('off')

    # Flattened visualization
    if flattened is not None:
        axes[1].plot(flattened.numpy())
        axes[1].set_title('Flattened to 1D Vector\n(4,096 values)', fontweight='bold')
        axes[1].set_xlabel('Pixel index')
        axes[1].set_ylabel('Pixel value')

    # Show what's lost
    axes[2].text(0.5, 0.5, '❌ Spatial Information\nLOST!\n\n'
                 'Pixels that were neighbors\nare now far apart in the vector',
                 ha='center', va='center', fontsize=14,
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()

    print(f"Original shape: {sample_img.shape}")
    print(f"Flattened shape: {flattened.shape if flattened is not None else 'N/A'}")
    print(f"\n🤔 Think about: Does the network know which pixels were next to each other?")

## Step 2.2: Design the MLP Architecture

**TODO:** Create an MLP classifier for images.

**Architecture:**
```
Input: 4,096 pixels (flattened 64×64 image)
Hidden 1: 128 neurons + ReLU
Hidden 2: 64 neurons + ReLU
Output: 2 classes (Normal vs Pneumonia)
```

In [ ]:
# TODO: Define the MLP classifier
# This is VERY similar to Lab 2, but with different input size!

class ChestXRayMLP(nn.Module):
    def __init__(self):
        super(ChestXRayMLP, self).__init__()

        # TODO: Define layers
        # Input size: 64*64 = 4096
        # Hint: self.fc1 = nn.Linear(4096, 128)
        #       self.relu = nn.ReLU()
        #       self.fc2 = nn.Linear(128, 64)
        #       self.fc3 = nn.Linear(64, 2)





    def forward(self, x):
        # TODO: Implement forward pass
        # IMPORTANT: First flatten the image!
        # Hint: x = x.view(x.size(0), -1)  # Flatten to (batch_size, 4096)
        #       x = self.fc1(x)
        #       x = self.relu(x)
        #       ...





        pass

# TODO: Create model instance
model = None  # ChestXRayMLP()



# Print model info
if model:
    print("Model Architecture:")
    print("="*60)
    print(model)
    print("="*60)

    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    print(f"\nTotal parameters: {total_params:,}")
    print(f"Compare to Lab 2: ~300 parameters")
    print(f"This model is {total_params/300:.0f}× larger!")

    # Test forward pass
    test_input = torch.randn(1, 1, 64, 64)
    test_output = model(test_input)
    print(f"\nTest: Input {test_input.shape} → Output {test_output.shape}")

## 🎉 Part 2 Complete!

**Checkpoint:**
- ✅ Understand image flattening
- ✅ MLP architecture defined
- ✅ MUCH larger than Lab 2 (due to 4,096 inputs!)

**Think about:** Why does flattening seem problematic?

---

---

# 🔄 PART 3: Training & Evaluation (20 minutes)

**This should look VERY familiar from Lab 2!**

---

## Step 3.1: Training Setup

**TODO:** Set up loss, optimizer, and device (same as Lab 2!).

In [ ]:
# TODO: Define loss function
criterion = None  # nn.CrossEntropyLoss()


# TODO: Define optimizer
optimizer = None  # optim.Adam(model.parameters(), lr=0.001)


# TODO: Set device
device = None  # torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# TODO: Move model to device
# model = model.to(device)


# Set epochs
num_epochs = 10  # Fewer epochs since images are slower to train

print("Training Setup:")
print("="*50)
print(f"Device: {device}")
print(f"Loss: {criterion}")
print(f"Optimizer: {optimizer.__class__.__name__ if optimizer else 'N/A'}")
print(f"Epochs: {num_epochs}")

## Step 3.2: Training Loop

**TODO:** Implement training loop (copy from Lab 2 and adapt!).

In [ ]:
# TODO: Implement training loop
# This is IDENTICAL to Lab 2 - you can reuse that code!

train_losses = []
val_losses = []
val_accuracies = []

print("Starting training...\n")

for epoch in range(num_epochs):
    # TODO: Training phase
    # Hint: Same as Lab 2!
    # 1. model.train()
    # 2. Loop through train_loader
    # 3. optimizer.zero_grad()
    # 4. Forward pass
    # 5. Compute loss
    # 6. loss.backward()
    # 7. optimizer.step()






    # TODO: Validation phase
    # Hint: Same as Lab 2!
    # 1. model.eval()
    # 2. with torch.no_grad()
    # 3. Loop through val_loader
    # 4. Compute accuracy





    pass

print("✓ Training complete!")

## Step 3.3: Plot Training Curves

In [ ]:
# TODO: Plot training curves (same as Lab 2!)

if len(train_losses) > 0:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Loss curves
    ax1.plot(train_losses, label='Training Loss', linewidth=2)
    ax1.plot(val_losses, label='Validation Loss', linewidth=2)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training Progress')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Accuracy curve
    ax2.plot(val_accuracies, label='Validation Accuracy', color='green', linewidth=2)
    ax2.axhline(y=50, color='red', linestyle='--', label='Random Guess')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.set_title('Validation Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"Final validation accuracy: {val_accuracies[-1]:.2f}%")

## Step 3.4: Evaluate on Test Set

**TODO:** Compute medical metrics on test set.

In [ ]:
# TODO: Evaluate on test set (same as Lab 2!)
# Compute: accuracy, precision, sensitivity, specificity, confusion matrix

if model:
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            # TODO: Get predictions



            pass

    # TODO: Compute metrics
    # accuracy = accuracy_score(all_labels, all_preds)
    # precision = precision_score(all_labels, all_preds)
    # sensitivity = recall_score(all_labels, all_preds)
    # cm = confusion_matrix(all_labels, all_preds)




    # Print results
    print("\nTest Set Results:")
    print("="*50)
    # print(f"Accuracy: {accuracy*100:.2f}%")
    # print(f"Precision: {precision*100:.2f}%")
    # print(f"Sensitivity: {sensitivity*100:.2f}%")

## 🎉 Part 3 Complete!

**How did we do?**

Compare to Lab 2:
- Lab 2 (diabetes, tabular): ~76% accuracy
- Lab 3 (pneumonia, images): ~??% accuracy

**Interesting, right?** 🤔

---

---

# 🤔 PART 4: Reflection & Analysis (20 minutes)

**The most important part: THINKING about what we learned!**

---

## Reflection Question 1: Performance Comparison

**Compare your results:**

| Metric | Lab 2 (Diabetes) | Lab 3 (Pneumonia) | Difference |
|--------|------------------|-------------------|------------|
| Input Features | 8 | 4,096 | 512× more |
| Parameters | ~300 | ~500,000 | 1,600× more |
| Accuracy | ~76% | ??% | ??% |
| Training Time/Epoch | Fast | Slower | ?? |

**Question:** Why might we get WORSE accuracy despite having MORE features and a BIGGER model?

**Your answer:**

_[Write your thoughts here]_

---

## Reflection Question 2: The Flattening Problem

When we flatten a 64×64 image:

```
Before:        After:
[row 0]        [pixel 0, pixel 1, pixel 2, ..., pixel 4095]
[row 1]        
[row 2]        Pixels that were neighbors
 ...           are now far apart!
[row 63]
```

**Questions to think about:**

1. **Does the MLP know that pixel 0 and pixel 1 were next to each other?**
   - Your answer: _[Yes/No and why]_

2. **In a chest X-ray, what matters more: individual pixel values OR patterns of nearby pixels?**
   - Your answer: _[Individual values / Patterns / Both?]_

3. **Example:** The pattern for "lung opacity" (pneumonia indicator) involves a cluster of bright pixels. Does our MLP naturally detect clusters?
   - Your answer: _[Yes/No and why]_

---

## Reflection Question 3: What's Missing?

**When a radiologist looks at a chest X-ray, they look for:**
- Shapes (heart silhouette, lung boundaries)
- Textures (smooth vs. patchy)
- Relative positions (is the opacity in the upper or lower lobe?)
- Edges (clear vs. fuzzy boundaries)

**Question:** Our MLP treats each pixel independently. What medical information might we be losing?

**Your answer:**

_[Write specific examples of spatial information that flattening destroys]_

---

## Reflection Question 4: If You Were Designing a Better Architecture...

**Imagine you could design a network specifically for images. What would you want it to do differently?**

Some ideas to consider:
- Process nearby pixels together?
- Look for local patterns (edges, shapes)?
- Understand that shifting the image slightly doesn't change the diagnosis?
- Build up understanding from small patterns → larger patterns?

**Your ideas:**

_[Write your thoughts on what would make a better image classifier]_

**Hint:** We'll explore solutions to these problems in the coming weeks! 😉

---

## Bonus Exploration: Visualize What the Model Learned

**TODO (Optional):** Look at some misclassified examples.

In [ ]:
# Optional: Show examples of correct and incorrect predictions
# This helps understand where the model struggles

# TODO: Get predictions and find misclassified images
# Show side-by-side: correct predictions vs wrong predictions

# Your exploration code here (optional):


---

## 🎓 Lab 3 Summary

### What We Accomplished:

✅ **Applied Lab 2 knowledge** to a new domain (images!)  
✅ **Loaded and visualized** medical images  
✅ **Built an MLP** for image classification  
✅ **Trained and evaluated** with medical metrics  
✅ **Discovered limitations** of MLPs for images  

### Key Insights:

1. **Flattening loses spatial information** - crucial for images!
2. **More features ≠ better performance** - if architecture is wrong
3. **Images are different from tabular data** - need different approaches
4. **Medical imaging is challenging** - but solvable with right tools!

### Coming Up:

- **Next week:** DICOM protocol, medical imaging modalities
- **Soon:** Better architectures for images (hint: CNNs!)
- **2-3 weeks:** Hospital visit to see MRI/CT machines!

---

## 📤 Submission Instructions

1. **Complete all TODOs** in this notebook
2. **Answer reflection questions** thoughtfully
3. **Save:** File → Download → .ipynb
4. **Name:** `Lab3_LastName1_LastName2.ipynb`
5. **Submit to Blackboard** by [deadline]

**Questions to discuss next class!** 💭

---